# Translation (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
# !apt install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and 

## Prepareing the data

In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.10k [00:00<?, ?B/s]

kde4.py:   0%|          | 0.00/4.25k [00:00<?, ?B/s]

The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/210173 [00:00<?, ? examples/s]

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

Let's split the data in train -test(validate)

In [4]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [5]:
# rename test to validation
split_datasets["validation"] = split_datasets.pop("test")

In [6]:
split_datasets["train"][1]["translation"]

{'en': 'Default to expanded threads',
 'fr': 'Par défaut, développer les fils de discussion'}

In [10]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint)
translator("Things are going to be tough now on, it's going to be a long night")

Device set to use cuda:0


[{'translation_text': 'Ça va être dur maintenant, ça va être une longue nuit.'}]

In [8]:
split_datasets["train"][172]["translation"]

{'en': 'Unable to import %1 using the OFX importer plugin. This file is not the correct format.',
 'fr': "Impossible d'importer %1 en utilisant le module d'extension d'importation OFX. Ce fichier n'a pas un format correct."}

In [9]:
translator(
    "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
)

[{'translation_text': "Impossible d'importer %1 en utilisant le plugin d'importateur OFX. Ce fichier n'est pas le bon format."}]

## Processing the data

- convert text to token IDs
- we need to tokenize both input and output

In [11]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

 - we need to ensure that the tokenizer processes the targets in the output language.
 - we can do this by passing the targets to the text_targets argument of the tokenizer’s __call__ method.

In [22]:
en_sentence = split_datasets["train"][4]["translation"]["en"]
fr_sentence = split_datasets["train"][4]["translation"]["fr"]

inputs = tokenizer(en_sentence, text_target=fr_sentence)
inputs

{'input_ids': [135, 607, 2054, 2, 3482, 10, 2843, 21048, 26, 67, 478, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [4322, 5, 30508, 2, 5, 4403, 11, 5, 6676, 27, 66, 478, 0]}

the output contains the` input IDs` associated with the English sentence, while the IDs associated with the French one are stored in the `labels` field

In [23]:
 print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

['▁Nombre', '▁de', '▁phrases', ',', '▁de', '▁mots', '▁et', '▁de', '▁lettres', '▁pour', '▁ce', '▁document', '</s>']


In [24]:
max_length = 128

def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

text is shorter so we have allocated only 128 as max length

In [25]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Map:   0%|          | 0/189155 [00:00<?, ? examples/s]

Map:   0%|          | 0/21018 [00:00<?, ? examples/s]

## Data collation


we can't use `DataCollatorWithPadding` because it pad only input (input IDs, attention mask, and token type IDs). But we want it to be padded to the maximum length encountered in the labels.
- padding labels should be -100, so they are ignored in the loss computation

We need use `DataCollatorForSeq2Seq`
- it takes the `tokenizer` used to preprocess the inputs, but it also takes the `model`. This is because this data collator will also be responsible for preparing the `decoder input IDs` which keep changing as per the model

In [26]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [27]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [37]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(3, 5)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [38]:
batch["labels"]

tensor([[ 4194,   442,     0,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100],
        [ 4322,     5, 30508,     2,     5,  4403,    11,     5,  6676,    27,
            66,   478,     0]])

In [39]:
batch["decoder_input_ids"]

tensor([[59513,  4194,   442,     0, 59513, 59513, 59513, 59513, 59513, 59513,
         59513, 59513, 59513],
        [59513,  4322,     5, 30508,     2,     5,  4403,    11,     5,  6676,
            27,    66,   478]])

In [43]:
 split_datasets["train"][4]["translation"]["fr"]


'Nombre de phrases, de mots et de lettres pour ce document'

In [40]:
 print(tokenizer.convert_ids_to_tokens(batch["labels"][1]))

['▁Nombre', '▁de', '▁phrases', ',', '▁de', '▁mots', '▁et', '▁de', '▁lettres', '▁pour', '▁ce', '▁document', '</s>']


In [41]:
 print(tokenizer.convert_ids_to_tokens(batch["decoder_input_ids"][1]))

['<pad>', '▁Nombre', '▁de', '▁phrases', ',', '▁de', '▁mots', '▁et', '▁de', '▁lettres', '▁pour', '▁ce', '▁document']


## Metrics

**BLEU:** it mesause precision on 1-gran, 2-gram,3-gram, 4-gram. and then mean it to calculate `score.`
- higher the better.
- It does not measure the intelligibility or grammatical correctness of the model’s generated outputs, it uses statistical rules  
- negative of BLEU is that it expects the text to already be tokenized, which makes it difficult to compare scores between models that use different tokenizers. SO we use ScareBLEU

In [44]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.0 MB/s eta 0:00:00


In [45]:
import evaluate

metric = evaluate.load("sacrebleu")

In [46]:
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [47]:
predictions = ["This This This This"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 1.683602693167689,
 'counts': [1, 0, 0, 0],
 'totals': [4, 3, 2, 1],
 'precisions': [25.0, 16.666666666666668, 12.5, 12.5],
 'bp': 0.10539922456186433,
 'sys_len': 4,
 'ref_len': 13}

In [48]:
predictions = ["This plugin"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 0.0,
 'counts': [2, 1, 0, 0],
 'totals': [2, 1, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 0.004086771438464067,
 'sys_len': 2,
 'ref_len': 13}

To write it in the function

In [49]:
tokenizer

MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-fr', vocab_size=59514, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	59513: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [50]:
tokenizer.pad_token_id

59513

In [51]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

## Fine-tuning the model


- the model will use the `decoder_input_ids` with an **attention mask** ensuring it does not use the tokens after the token it’s trying to predict
- decoder performs inference by predicting tokens one by one which we can initiate using `predict_with_generate=True`

**Try** TrainingArguments

In [52]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    #f"marian-finetuned-kde4-en-to-fr",
    eval_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
)

- We don’t set any regular evaluation, as evaluation takes a while; we will just evaluate our model once before training and after.
- We set `fp16=True`, which speeds up training on modern GPUs.
- We set `predict_with_generate=True`, as discussed above.

In [53]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-53-1468824c6e77>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [54]:
trainer.evaluate(max_length=max_length)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kanav608 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'eval_loss': 1.6966997385025024,
 'eval_model_preparation_time': 0.0039,
 'eval_bleu': 17.119091541005794,
 'eval_runtime': 1424.6825,
 'eval_samples_per_second': 14.753,
 'eval_steps_per_second': 0.231}

In [66]:
trainer.train()

Step,Training Loss
500,1.285700
1000,1.215000
1500,1.188300
2000,1.132800
2500,1.104700
3000,1.074600
3500,1.042400
4000,1.028000
4500,1.037100
5000,1.009600


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=17736, training_loss=0.9310185703867774, metrics={'train_runtime': 3296.4712, 'train_samples_per_second': 172.143, 'train_steps_per_second': 5.38, 'total_flos': 1.143155222544384e+16, 'train_loss': 0.9310185703867774, 'epoch': 3.0})

In [67]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 0.8550083637237549,
 'eval_model_preparation_time': 0.0039,
 'eval_bleu': 35.47640876055829,
 'eval_runtime': 1408.4848,
 'eval_samples_per_second': 14.922,
 'eval_steps_per_second': 0.234,
 'epoch': 3.0}

## A custom training loop

### 1. Preparing everything for training


In [68]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

Next we reinstantiate our model, to make sure we’re not continuing the fine-tuning from before but starting from the pretrained model again:

In [69]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [70]:
# Optmizer
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [71]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

let's use its length to compute the number of training steps

In [72]:
 len(train_dataloader)

23645

In [73]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

### 2. Training loop


To simplify its evaluation part, we define this `postprocess()` function that takes predictions and labels and converts them to the lists of strings our metric object will expect:

In [74]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

**Base Model:** `model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint) `

**Wrapped Model:**` When we use libraries like 🤗 Accelerate to make your code run on multiple GPUs, TPUs, or with mixed precision, we often "wrap" our model.

`model, optimizer, train_dataloader = accelerator.prepare(model, optimizer, train_dataloader)`

**Why Do We Need a Wrapped Model?**
- `Device Management:` The wrapped model is automatically placed on the correct device (CPU, GPU, TPU).

- `Distributed Training:` If you use multiple GPUs or machines, the wrapped model handles communication between them.

- `Mixed Precision:` The wrapper can enable faster computation using mixed precision (float16).

**Why "Unwrap" the Model?**
Some methods (like .generate()) are not available or not correctly implemented on the wrapped model

We may have padded the **inputs and labels** to different shapes, so we use `accelerator.pad_across_processes()` to make the predictions and labels the same shape before calling the `gather()` method. If we don’t do this, the evaluation will either error out or hang forever.

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # Save and upload
    # accelerator.wait_for_everyone()
    # unwrapped_model = accelerator.unwrap_model(model)
    # unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    # if accelerator.is_main_process:
    #     tokenizer.save_pretrained(output_dir)
    #     repo.push_to_hub(
    #         commit_message=f"Training in progress epoch {epoch}", blocking=False
    #     )

  0%|          | 0/70935 [00:00<?, ?it/s]